In [3]:
from bs4 import BeautifulSoup
import requests
import requests_cache
import pandas as pd

In [4]:
requests_cache.install_cache('request_cache')

In [5]:
url = "https://basketballmonster.com/PlayerRankings.aspx"
r = requests.get(url)
soup = BeautifulSoup(r.text, "html.parser")
header = soup.find("thead")
header.find_all("th")[:3]

[<th class="nowrap">Round</th>,
 <th class="nowrap"><a class="" href='javascript:__doPostBack("SortRank","")' id="SortRank">Rank</a></th>,
 <th class="nowrap"><a class="tooltip-control" data-placement="top" data-toggle="tooltip" href='javascript:__doPostBack("SortTotalValue","")' id="SortTotalValue" title="Our Value for this player">Value</a></th>]

In [6]:
cols = [col.text if not col.a else col.a.text for col in header.find_all("th")]
cols[:5]

['Round', 'Rank', 'Value', 'Name', 'Inj']

In [7]:
# `<tbody></tbody>` Tag is loaded with Javascript so need to switch tooling here
from requests_html import HTMLSession, AsyncHTMLSession

In [8]:
import asyncio
if asyncio.get_event_loop().is_running(): # Only patch if needed (i.e. running in Notebook, Spyder, etc)
    import nest_asyncio
    nest_asyncio.apply()

In [9]:
asession = AsyncHTMLSession()
r = await asession.get('https://basketballmonster.com/PlayerRankings.aspx')

In [10]:
await r.html.arender()

In [11]:
resp = r.html

In [12]:
type(resp)

requests_html.HTML

In [13]:
tbody = resp.find('tbody')[0]

In [14]:
tbody.attrs

{}

In [15]:
from selenium import webdriver
from selenium.webdriver import ChromeOptions

In [16]:
chromedriver = "/Users/samdillard/Downloads/chromedriver"
options = ChromeOptions()
options.add_argument('--headless')
driver = webdriver.Chrome(chromedriver, options=options)

In [17]:
driver.get('https://basketballmonster.com/PlayerRankings.aspx')

In [18]:
from selenium.webdriver.support.wait import WebDriverWait

In [19]:
wait = WebDriverWait

In [20]:
button = driver.find_element_by_xpath("//input[@value='Export to Excel']")

In [21]:
type(button)

selenium.webdriver.remote.webelement.WebElement

In [22]:
try:
    button.click()
except:
    print("fail")

## Temporarily using download CSV

In [23]:
bbm = pd.read_csv("~/Downloads/BBM_PlayerRankings1.csv")

In [24]:
bbm.head(3)

,Round,Rank,Y!Adp9,Y!ORank,Y!%,Value,Name,Inj,Team,Pos,...,3V,rV,aV,sV,bV,fg%V,ft%V,orV,2dV,atoV
0,1,1,3.0,4.0,100.0,1.060874,James Harden,NaN,HOU,G,...,2.842392,0.155915,1.827560,2.343359,0.372938,-1.118048,2.509516,-0.382300,0.638844,-0.597206
1,1,2,4.0,2.0,98.0,0.931042,Karl-Anthony Towns,NaN,MIN,C,...,1.738033,1.766628,0.357258,-0.094398,0.948741,0.838063,0.167021,1.401427,2.386989,-1.033760
2,1,3,2.0,1.0,100.0,0.918482,Anthony Davis,NaN,LAL,F,...,-0.292445,1.193283,-0.176123,1.334654,2.954420,0.691253,1.358957,0.935514,1.334909,-0.936635


In [25]:
bbm.columns

Index(['Round', 'Rank', 'Y!Adp9', 'Y!ORank', 'Y!%', 'Value', 'Name', 'Inj',
       'Team', 'Pos', 'g', 'm/g', 'p/g', '3/g', 'r/g', 'a/g', 's/g', 'b/g',
       'fg%', 'fga/g', 'ft%', 'fta/g', 'or/g', '2d/g', 'ato', 'a+to/g', 'pV',
       '3V', 'rV', 'aV', 'sV', 'bV', 'fg%V', 'ft%V', 'orV', '2dV', 'atoV'],
      dtype='object')

In [26]:
bbm_value = bbm.drop(labels=['Inj','Team','p/g','3/g','r/g','a/g','s/g','b/g','2d/g','a+to/g'], axis=1)
type(bbm_value.columns)

pandas.core.indexes.base.Index

In [27]:
bbm_final = bbm_value[[i for i in list(bbm_value.columns) if not i.startswith("Y!")]]

In [28]:
bbm_final.head(5)

,Round,Rank,Value,Name,Pos,g,m/g,fg%,fga/g,ft%,...,3V,rV,aV,sV,bV,fg%V,ft%V,orV,2dV,atoV
0,1,1,1.060874,James Harden,G,68,36.507843,0.443857,22.264706,0.865000,...,2.842392,0.155915,1.827560,2.343359,0.372938,-1.118048,2.509516,-0.382300,0.638844,-0.597206
1,1,2,0.931042,Karl-Anthony Towns,C,35,33.913810,0.508039,17.771429,0.796460,...,1.738033,1.766628,0.357258,-0.094398,0.948741,0.838063,0.167021,1.401427,2.386989,-1.033760
2,1,3,0.918482,Anthony Davis,F,62,34.377151,0.502737,17.677419,0.845714,...,-0.292445,1.193283,-0.176123,1.334654,2.954420,0.691253,1.358957,0.935514,1.334909,-0.936635
3,1,4,0.861658,LeBron James,G,67,34.571891,0.493477,19.447761,0.692913,...,0.722541,0.640893,3.075108,0.508174,-0.252520,0.487852,-1.460370,-0.445039,2.391105,2.230345
4,1,5,0.839110,Damian Lillard,G,67,37.591542,0.461877,20.358209,0.890977,...,2.558937,-0.713844,2.074453,0.223723,-0.631296,-0.465930,2.254654,-0.950673,0.518072,2.009705


In [33]:
bbm_final.sort_values(by=['aV','ft%V','3V'], ascending=False).iloc[30:65].head(5)

,Round,Rank,Value,Name,Pos,g,m/g,fg%,fga/g,ft%,...,3V,rV,aV,sV,bV,fg%V,ft%V,orV,2dV,atoV
87,8,88,-0.148578,Derrick Rose,G,50,25.952333,0.490040,15.060000,0.870504,...,-0.545585,-1.415927,0.910636,-0.483340,-0.646447,0.297150,0.630204,-0.900425,-0.536590,0.684210
86,8,87,-0.143122,Tomas Satoransky,G,65,28.891026,0.430127,8.476923,0.876033,...,-0.463609,-0.850931,0.857632,0.689472,-1.031300,-0.608012,0.448117,-0.234277,-0.699954,1.306399
84,8,85,-0.135588,Eric Bledsoe,G,61,26.988525,0.473759,11.557377,0.789720,...,-0.242237,-0.566839,0.825458,-0.117352,-0.424167,-0.059829,0.023499,-0.691121,-0.764472,0.677166
135,12,136,-0.336811,Jeff Teague,G,59,24.818644,0.435583,8.288136,0.873096,...,-0.711432,-1.420948,0.752496,-0.631007,-0.642760,-0.525996,0.781741,-0.987631,-0.677300,1.186624
102,9,103,-0.258368,Joe Ingles,F,72,29.674537,0.444846,7.680556,0.787234,...,0.479736,-0.868622,0.733974,-0.203518,-0.949573,-0.380037,-0.003846,-1.075106,-0.721612,1.149110


In [30]:
# new_df = bbm_final.reindex(bbm_final['Name'], axis=0)
players = bbm_value['Name']
new_df = pd.DataFrame(bbm_value, index=players)


In [31]:
def place(player, bbm_final):
#     team_df = team_df.append(bbm_final[(bbm_final['Name'] == f"{player}")])
    index = bbm_final[bbm_final.Name == f"{player}"][0]
    print(index)
    bbm_final.drop(bb)


In [32]:
# place("James Harden", bbm_final)
index = 4
new_df = bbm_final[bbm_final.iloc[index])]


SyntaxError: closing parenthesis ')' does not match opening parenthesis '[' (<ipython-input-32-9d5fba950e49>, line 3)

In [ ]:
index = bbm_final[bbm_final.Name == "Damian Lillard"].index.values

In [ ]:
index[0]